In [1]:
import pandas as pd
import numpy as np
import datetime
import json
import time
import os
import music21 as m21
from music21.key import Key
from music21.interval import Interval
from music21.pitch import Pitch
from music21.converter import parseFile

MSCORE_PATH = '/usr/bin/mscore'
LILYPOND_PATH = '/usr/bin/lilypond'

settings = m21.environment.UserSettings()

settings['musicxmlPath'] = MSCORE_PATH
settings['musescoreDirectPNGPath'] = MSCORE_PATH

%load_ext music21.ipython21

In [2]:
import logging
import imp

imp.reload(logging)
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.DEBUG)

logger = logging.getLogger()
logger.handlers.clear()

#file handler
log_filename = datetime.datetime.now().strftime('%Y%m%d_%H%M%S_processing.log')
fh = logging.FileHandler('../logs/' + log_filename)
fh.setLevel(logging.DEBUG)
logger.addHandler(fh)

#console handler
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)
logger.addHandler(ch)

logger.debug('debug')
logger.info('info')

info


In [3]:
def extract_filename(row):
    filename, url = parse_descriptor(row['files'])
    return pd.Series({ 'filename': filename, 'url': url })

In [4]:
def parse_descriptor(files_string):
    try:
        descriptors = json.loads(files_string.replace("'", '"'))
    except ValueError as e:
        logger.error('... files bad descriptor: %s', files_string)
        raise
        
    if(len(descriptors) > 1):
        logger.warn('Descriptor contains several files: %s', files_string)
        
    filename = os.path.basename(descriptors[0]['path'])
    filename = filename.replace('.krn&f=midi', '.mid')
    url = descriptors[0]['url']
    
    return filename, url

In [5]:
def process_file_descriptors(dataframe):
    descriptor_data = pd.DataFrame(dataframe.apply(extract_filename, axis=1))
    return pd.concat([dataframe, descriptor_data], axis=1)

In [6]:
def load_dataframe(metadata_path, files_path):
    dataframe = pd.read_csv(metadata_path, dtype=str)
    dataframe['input_dir'] = files_path
    dataframe = dataframe[~dataframe['files'].isnull()]
    dataframe = process_file_descriptors(dataframe)
    dataframe = dataframe.fillna('')
    return dataframe[dataframe['filename'].str.endswith('.mid', na=False)]

In [7]:
kern_data = load_dataframe('../data/kernscores.csv', '../data/files_kernscores/')

In [8]:
matched_instruments = [
    'Piano', 
    'Harpsichord,Piano', 
    'Harpsichord,Piano,Clavichord',
    'Piano Duet',
    'Harpsichord,Clavichord,Piano',
    'Piano,Harpsichord'
]

mutopia_data = load_dataframe('../data/mutopia.csv', '../data/files_mutopia/') 
mutopia_data = mutopia_data[mutopia_data['instruments'].isin(matched_instruments)]

In [9]:
pianomidi_data = load_dataframe('../data/pianomidi.csv', '../data/files_pianomidi/') 

In [10]:
def call_safe(func, args, default=None):
    try:
        return func(*args)
    except Exception as e:
        logger.exception(e)
        return default

In [11]:
def normalize(score, key_str):
    key = m21.key.Key(key_str.split()[0])
    
    if key.mode == 'major':
        i = Interval(key.tonic, Pitch('C'))

    elif key.mode == 'minor':
        i = Interval(key.tonic, Pitch('A'))
        
    score.transpose(i, inPlace=True)

In [12]:
def update_metadata(score, score_metadata):
    if score.metadata is None:
        score.metadata = m21.stream.metadata.Metadata()
        
    score.metadata.composer = score_metadata['composer']
    score.metadata.title = score_metadata['name']
    score.metadata.movementName = score_metadata['name']
    
    try:
        score.metadata.date = score_metadata['year']
    except:
        pass

In [13]:
def extract_time_signatures(score):
    for part in score.parts:
        time_signatures = [m.timeSignature for m in part.getElementsByClass(m21.stream.Measure) if m.timeSignature is not None]
        if time_signatures:
            return [ts.ratioString for ts in time_signatures]
    
    return [None]

In [ ]:
# Hard way to extract duration:
# metronome_data = pd.DataFrame(score.metronomeMarkBoundaries(), columns=['start', 'end', 'tempo'])
# metronome_data['duration'] = metronome_data['end'] - metronome_data['start']
# metronome_data['bpm'] = metronome_data['tempo'].map(lambda x: x.number)
# metronome_data['weighted_duration'] = metronome_data['bpm'] * metronome_data['duration']
# tempo = metronome_data['weighted_duration'].sum() /  metronome_data['duration'].sum()
# tempo

In [14]:
def extract_stats(score):
    key_signature = score.analyze('key')
    primary_ts, *alternative_ts = call_safe(extract_time_signatures, [score], default=[None])
    duration = call_safe(lambda s: max([x['endTimeSeconds'] for x in s.secondsMap]), [score])      
    tempo = call_safe(lambda s: s.metronomeMarkBoundaries()[0][2].number, [score]) 
        
    
    return {
        'key': str(key_signature),
        'primary_time_signature': primary_ts,
        'secondary_time_signatures': alternative_ts,
        'duration': duration,
        'tempo': tempo
    }

In [15]:
def set_extension(filename, new_ext):
    base, ext = os.path.splitext(filename)
    return base + '.' + new_ext

In [16]:
def parse_file(input_path):
    _, ext = os.path.splitext(input_path)
    if ext != '.xml' and ext != '.mxl':
        tmp_path = input_path + '.tmp.xml'
        subprocess.call('mscore "{0}" -o "{1}"'.format(input_path, tmp_path), shell=True)
        return m21.converter.parseFile(tmp_path)
    else:
        return m21.converter.parseFile(input_path)

In [ ]:
def convert_to_abc():
    

In [17]:
import os
import json
import subprocess
    
def process_row(row, output_dir, global_time_started):
    hasError = False
    output_path = None
    key_signature = None
    primary_ts = None
    alternative_ts = None
    url = None
    score_stats = {
        'key': None,
        'primary_time_signature': None,
        'secondary_time_signatures': None,
        'duration': None,
        'tempo': None
    }
    
    try:
        logger.debug('Processing %s...', row['name'])
        time_started = time.time()
        
        input_path = os.path.join(row['input_dir'], row['filename'])
        output_path = os.path.join(output_dir, set_extension(row['filename'], 'xml'))
        
        score = parse_file(input_path)
        logger.debug('\t...file parsed')
        
        score_stats = extract_stats(score)
        logger.debug('\t...stats extractes: %s', json.dumps(score_stats))
    
        normalize(score, score_stats['key'])
        update_metadata(score, row)
        logger.debug('\t...score normalized and updated')
        
        score.write('musicxml', output_path)
        logger.debug('\t...file processed. Processing time: %s', time.time() - time_started)
        logger.info('Processed file %i. Time elapsed %s', row.name, time.time() - global_time_started)
        
    except Exception as e:
        logger.error('Exception occured')
        logger.debug('Exception details: ', exc_info=e)
        hasError = True
        
    return pd.Series({
            'genre': row['genre'],
            'input_dir': row['input_dir'],
            'details_url': row['details_url'],
            'filename_mid' : row['filename'],
            'name': row['name'],
            'year': row['year'],
            'composer': row['composer'],
            'filename_xml': output_path,
            'source_url': url,
            'key': score_stats['key'],
            'primary_time_signature': score_stats['primary_time_signature'],
            'secondary_time_signatures': score_stats['secondary_time_signatures'],
            'duration': score_stats['duration'],
            'tempo': score_stats['tempo'],
            'hasError': hasError
        })

In [18]:
import time

def process_dataframe(dataframe, output_dir):
    time_started = time.time()
    total_rows = len(dataframe.index)
    logger.info('Processing files in dataframe. Total items: %i', total_rows)
    result = dataframe.apply(lambda row: process_row(row, '../data/output/', time_started), axis=1)
    logger.info('Finished processing dataframe')
        
    return result

In [20]:
dataset = pd.concat([mutopia_data, kern_data, pianomidi_data])
dataset.reset_index(inplace=True)
del dataset['index']
dataset.describe()

,catalogue_code,composer,details_url,file_urls,filename,files,genre,input_dir,instruments,key,name,tags,url,year
count,1171,1171,1171,1171,1171,1171,1171,1171,1171,1171,1171,1171,1171,1171
unique,302,100,864,1171,1171,1171,13,3,7,26,981,1,1171,151
top,,J. S. Bach,http://www.piano-midi.de/chopin.htm,http://www.mutopiaproject.org/ftp/MozartWA/KV3...,1f69c791b88c81e64558f358786a8a46d1b68477.mid,[{'checksum': '620895a014c1f7913dc3ecada3f86ff...,Baroque,../data/files_mutopia/,Piano,,Prelude,,http://www.mutopiaproject.org/ftp/MozartWA/KV3...,
freq,423,114,48,1,1,1,237,573,748,1075,53,1171,1,359


In [21]:
metadata = pd.DataFrame()
CHUNK_SIZE = 100

for i in range(0, len(dataset.index), CHUNK_SIZE):
    chunk = dataset[i:i+CHUNK_SIZE]
    logger.info('processing chunk {0} to {1}'.format(i, i+CHUNK_SIZE))
    metadata = pd.concat([metadata, process_dataframe(chunk, '../data/output/xml')])
    metadata.to_csv('metatada.backup.{0}.csv'.format(i))
    logger.info('...done')

processing chunk 0 to 100
Processing files in dataframe. Total items: 100
Exception occured
Exception occured


NameError: ("name 'rowp' is not defined", 'occurred at index 0')

In [ ]:
def get_filename_abc(input_filename):
    output_file, _ = os.path.splitext(os.path.basename(input_filename))
    return '../data/output/abc/' +  output_file + '.abc'

In [68]:
def get_semitones_to_cdur(key):
    try:
        if key.mode == 'major':
            interval = m21.interval.Interval(key.tonic, m21.pitch.Pitch('C'))
            semitones = interval.semitones if interval.semitones < 6 else 12 - interval.semitones
            return semitones
        if key.mode == 'minor':
            interval = m21.interval.Interval(key.tonic, m21.pitch.Pitch('A'))
            semitones = interval.semitones if interval.semitones < 6 else 12 - interval.semitones
            return semitones
    except:
        pass
    
    logger.error('Bad key: %s', key)
    return None

In [84]:
import subprocess

def normalize_abc(filename):
    basename = os.path.basename(filename)
    
    if not os.path.isfile(filename):
        logger.warning()
        return
    
    score = m21.converter.parseFile(filename)
    key_signature = score.analyze('key')
    semitones = get_semitones_to_cdur(key_signature)
    
    command = 'abc2abc "{0}" -t {1} > "../output/abc_transposed/{2}"'.format(filename, semitones, basename)
    subprocess.call(command, shell=True)

In [88]:
normalize_abc("../output/abc/537688d173f52c3a7fd3371a57cdec22d5661285.abc")

In [90]:
! abc2abc "../output/abc/537688d173f52c3a7fd3371a57cdec22d5661285.abc" -t 4 > "../output/abc_transposed/537688d173f52c3a7fd3371a57cdec22d5661285.abc"

/usr/bin/sh: ../output/abc_transposed/537688d173f52c3a7fd3371a57cdec22d5661285.abc: No such file or directory


In [91]:
for index, row in metadata.iterrows():
    filename = row['filename_abc']
    normalize_abc(filename)
    if index % 100 == 0:
        print('\rProcessed {0} files out of {1}'.format(index, len(metadata.index)))

Processed 0 files out of 2187
Processed 100 files out of 2187
Processed 200 files out of 2187
Processed 300 files out of 2187
Processed 400 files out of 2187
Processed 500 files out of 2187
Processed 600 files out of 2187
Processed 700 files out of 2187
Processed 800 files out of 2187
Processed 900 files out of 2187
Processed 1000 files out of 2187
Processed 1100 files out of 2187


TypeError: warning() missing 1 required positional argument: 'msg'

In [ ]:
# checks
metadata = metadata[~metadata['hasError']]   

if not all(metadata['filename_abc'].map(lambda fname: os.path.isfile(fname))):
    logger.error('missing abc files!')
    
if not all(metadata['filename_xml'].map(lambda fname: os.path.isfile(fname))):
    logger.error('missing xml files!')
    
if not all(metadata['filename_mid'].map(lambda fname: os.path.isfile(fname))):
    logger.error('missing mid files!')

In [5]:
metadata = pd.read_csv('../output/metadata.csv')

In [15]:
metadata['time_signature'] = None
metadata['secondary_time_signatures'] = None

In [16]:
def parse_time_signature(row):
    if row['time_signature'] is not None:
        return pd.Series({
            'time_signature': row['time_signature'],
            'secondary_time_signatures': row['secondary_time_signatures']
        })
    
    filename = row['filename_xml']
    score = m21.converter.parseFile(filename)
    primary_ts, *alternative_ts = call_safe(extract_time_signatures, [score], default=[None])
    logger.debug('%s: extracted primary time signature: %s', filename, primary_ts)
    return pd.Series({
            'time_signature': primary_ts,
            'secondary_time_signatures': '; '.join(alternative_ts)
        })

In [17]:
result = metadata.apply(parse_time_signature, axis=1)

In [20]:
head()

0    2/4
1    3/4
2    2/2
3    6/8
4    6/8
Name: primary_time_signature, dtype: object

In [23]:
del metadata['primary_time_signature']

In [25]:
del metadata['secondary_time_signatures']
del metadata['time_signature']
metadata = pd.concat([metadata, result], axis=1)

In [26]:
metadata.describe()

,duration,hasError,source_url,tempo,year,year_exact
count,2187.000000,2187,0,2187.000000,822.000000,2187
mean,223.170912,0,NaN,115.157293,1819.451338,0.275263
std,221.959010,0,NaN,42.066431,60.970446,0.446749
min,8.727273,False,NaN,18.000000,1720.000000,False
25%,90.000000,0,NaN,84.000000,1775.000000,0
50%,160.761450,0,NaN,112.000000,1828.000000,0
75%,272.497926,0,NaN,141.000000,1850.000000,1
max,2156.893319,False,NaN,300.000000,2008.000000,True


In [24]:
metadata[['filename_abc', 'key']].head()

,filename_abc,key
0,../output/abc/6e355a07d5531f372a2f7fb9ac084a12...,G major
1,../output/abc/cf0e37cd297b4b030a506f1e1e2c6373...,d minor
2,../output/abc/9235e4d8f77349089d45f475fb852ab7...,C major
3,../output/abc/6ece453ca4108e48ea6e6d0fec699644...,D major
4,../output/abc/96b2c5072caf99d739a42bad844e11b6...,G major


In [23]:
metadata = pd.read_csv('../output/metadata.csv', index_col=0)